Master Problem Objective: 

$max~\nu_{chemical}$   

Constraints:

$\sum S_{ij} * \nu_j = 0 $                                                

$LB*y_j \leq \nu_j \leq UB*y_j$                                          

$\sum (1-y_j) \leq k$


Inner Problem Objective:

$max~M*\nu_{biomass} $

Constraints:

$\sum S_{ij} * \nu_j = 0 $

$LB *\hat{y_j} \leq \nu_j \leq UB*\hat{y_j}$


In this escenario the values of $y$ are:

$y_j = 1$ if the reaction j is kept

$y_j = 0$ if the reaction j is knocked out

Therefore the constrs associated are:

Bounds:

$LB*y_j \leq v_j \leq UB*y_j$

Knapsack:

$\sum (1 - y_j) \leq k,~\forall j \in knockout$

Lazy cut:

$\nu_{inner-biomass}~\leq~vas_{bm-core} + M * \sum(1-\tilde{y_j}) *|\nu_j|$



In [1]:
# Libraries to import
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import csv

In [2]:
# Read the data from the csv files

UB = pd.read_csv('iJO1366_UB.csv',header=None)[0].tolist()

LB = pd.read_csv('iJO1366_LB.csv',header=None)[0].tolist()
 
rxn = pd.read_csv('iJO1366_rxn.csv',header=None)[0].tolist()

met = pd.read_csv('iJO1366_met.csv',header=None)[0].tolist()

b = pd.read_csv('iJO1366_b.csv',header=None)[0].tolist()

c = pd.read_csv('iJO1366_c.csv',header=None)[0].tolist()

S = pd.read_csv('iJO1366_S.csv',header=None).values

r = pd.read_csv('iJO1366_r.csv',header=None)[0].tolist() # if reaction i is reversible r[i] = 1

names = pd.read_csv('iJO1366_names.csv',header=None)[0].tolist()

In [3]:
vjc = pd.read_csv('iJO1366_vj_COBRA_sol.csv',header=None)[0].tolist()
fba = pd.read_csv('iJO1366_FBA.csv',header=None)[0].tolist()

In [4]:
# Changing the biological assumptions

# prespecified amount of glucose uptake 10 mmol/grDW*hr 'EX_glc__D_e' = -10 reaction 

LB[rxn.index('EX_glc__D_e')] = -10
UB[rxn.index('EX_glc__D_e')] = -10

# Unconstrained uptake routes for inorganic phosphate, sulfate and ammonia 
# 'EX_o2_e';'EX_pi_e';'EX_so4_e'; 'EX_nh4_e' index = 184 ; 199 ; 259 ; 169

LB[rxn.index('EX_o2_e')] = 0
LB[rxn.index('EX_pi_e')] = -1000
LB[rxn.index('EX_so4_e')] = -1000
LB[rxn.index('EX_nh4_e')] = -1000

#Enable secretion routes for acetate, carbon dioxide, ethanol, formate, lactate and succinate
# {'EX_ac_e';'EX_co2_e';'EX_etoh_e';'EX_for_e';'EX_lac__D_e';'EX_succ_e'} change in the upper bound 
# index = 87; 2; 340; 422; 91; 261

sec_routes = ['EX_ac_e','EX_co2_e','EX_etoh_e','EX_for_e','EX_lac__D_e','EX_succ_e']

for i in sec_routes:
    UB[(rxn.index(i))] = 1000
    
# Constrain the phosphotransferase system - 'b' means a change in both bounds to fix a flux
# 'GLCabcpp', -1000, 'l' - 'GLCptspp', -1000, 'l' - 'GLCabcpp', 1000, 'u' - 'GLCptspp', 1000, 'u' - 'GLCt2pp', 0, 'b'
# index 1287, 1291, 1287, 1291, 1293

LB[rxn.index('GLCabcpp')] = -1000
LB[rxn.index('GLCptspp')] = -1000
UB[rxn.index('GLCabcpp')] = 1000
UB[rxn.index('GLCptspp')] = 1000
LB[rxn.index('GLCt2pp')] = 0 
UB[rxn.index('GLCt2pp')] = 0

In [5]:
# List of set of reactions for Knockouts ie. only reactions in this set will be deleted

set_reaction = ['GLCabcpp', 'GLCptspp', 'HEX1', 'PGI', 'PFK', 'FBA', 'TPI', 'GAPD','PGK', 'PGM', 'ENO', 'PYK', 
'LDH_D', 'PFL', 'ALCD2x', 'PTAr', 'ACKr','G6PDH2r', 'PGL', 'GND', 'RPI', 'RPE', 'TKT1', 'TALA', 'TKT2', 'FUM',
'FRD2', 'SUCOAS', 'AKGDH', 'ACONTa', 'ACONTb', 'ICDHyr', 'CS', 'MDH','MDH2', 'MDH3', 'ACALD']

knockout = []

for i in set_reaction:
    knockout.append(rxn.index(i))

# succinate production and growth rate

rxn_target = ['EX_succ_e','EX_etoh_e','EX_for_e','EX_lac__D_e','EX_ac_e']
in_target = []

for i in rxn_target:
    in_target.append(rxn.index(i))


In [6]:
# Defining the index for the biomass 'BIOMASS_Ec_iJO1366_core_53p95M'

bm_core = rxn.index('BIOMASS_Ec_iJO1366_core_53p95M')

# Defining the index for the chemical target succinate 'EX_succ_e'

chemical = rxn.index('EX_succ_e')

k1 = rxn.index('PFL')
k2 = rxn.index('TALA')

In [7]:
c_sol = [1 for i in range(len(rxn))]
c_sol[k1] = 0
c_sol[k2] = 0

In [8]:
FBA_WT = fba[bm_core]

In [ ]:
k = 2
reactions = [i for i in range(len(rxn))]
metabolites = [i for i in range(len(met))]

def inner(yoj):
    global vis
    i = gp.Model()

    vi = i.addVars(range(len(rxn)),lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='vi')
    
    i.setObjective(2000*vi[bm_core], GRB.MAXIMIZE) 

    i.addConstrs((gp.quicksum(S[i,j] * vi[j] for j in reactions) == 0 for i in metabolites), name='IFFBA')

    i.addConstrs((LB[j]*yoj[j] <= vi[j] for j in reactions), name='iLB')
    i.addConstrs((vi[j] <= UB[j]*yoj[j] for j in reactions), name='iUB')
    
    i.optimize()

    if i.status == GRB.OPTIMAL:
        soi = i.getAttr('X',i.getVars())
        vis = soi.copy()
        
    elif i.status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
        vis = yoj
        vis[bm_core] = 2000
        
    return vis



def lazyctr(model,where):
    print(str(where == GRB.Callback.MIPSOL))
    if where == GRB.Callback.MIPSOL:
        voj = model.cbGetSolution(model._vars) #solutions
        yoj = model.cbGetSolution(model._varsy)
        keys = model._vars.keys()  #keys
        vjs = model._vars   #variables master's vj
        yjs = model._varsy  #variables master's yj
        eps = .00001
        vij = inner(yoj)
        
        if abs(vij[bm_core]-voj[bm_core]) >= eps:
            print('***CUT***')
            model.cbLazy(vij[bm_core] <= vjs[bm_core] + 2000*(gp.quicksum((1-yjs[j])*abs(vij[j]) for j in keys)))
            
            print('***Check 1***')
            print(str(where==GRB.Callback.MIPNODE))
            if where == GRB.Callback.MIPNODE:
                print('*** Check 2 ***')
                model.cbSetSolution(vjs, vij)
                model.cbSetSolution(yjs, yoj)
    

m = gp.Model()

v = m.addVars(range(len(rxn)), lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name= 'v')
y = m.addVars(range(len(rxn)), vtype=GRB.BINARY, name= 'y')

m.setObjective(1*v[chemical], GRB.MAXIMIZE)

m.addConstrs((gp.quicksum(S[i,j] * v[j] for j in reactions) == 0 for i in metabolites), name='OFFBA')

# Bounds and y's
m.addConstrs((y[j] == 1 for j in reactions if j not in knockout))

m.addConstrs((LB[j]*y[j] <= v[j] for j in reactions), name='LBy')
m.addConstrs((v[j] <= UB[j]*y[j] for j in reactions), name='UBy')

m.addConstr(sum(1-y[j] for j in knockout) == k, name='Knapsack') 

m.addConstr(v[bm_core] >= .5*FBA_WT, name='expected_growth') # the value for FBA_WT comes from the first FBA

#m.addConstr(y[k1]==0,name='COBRAsol1')
#m.addConstr(y[k2]==0,name='COBRAsol2')

m._vars = v
m._varsy = y
m.Params.lazyConstraints = 1

m.optimize(lazyctr)

s = m.Runtime

if m.status == GRB.OPTIMAL:
    sol = m.getAttr('X',m.getVars())
    sol1= sol.copy()
    yoj = sol1[len(rxn):]
    voj = sol1[:len(rxn)]

    #dfy = pd.DataFrame(data={'Cut final':yoj})
    #dfy.to_csv("iJO1366_refor_yoj_final_COBRA_PFL_TALA_1.csv",mode='a', sep=',',index=False)
    #dfv = pd.DataFrame(data={'col1':voj})
    #dfv.to_csv("iJO1366_refor_voj_final_COBRA_PFL_TALA_1.csv",mode='a', sep=',',index=False)
    
      
if m.status in (GRB.INFEASIBLE,GRB.INF_OR_UNBD,GRB.UNBOUNDED):
    m.computeIIS()
    #m.write('iJO1366_Bilevel_k2_succinate.ilp')
    if m.IISMinimal:
        print('IIS is minimal\n')
    else:
        print('IIS is not minimal\n')
        print('\nThe folllowing constraint(s) cannot be satisfied:')
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s'%c.constrName)

m = s/60

h = m/60

print('Run time in seconds:',' ', s)
print('Run time in minutes:',' ', m)
print('Run time in hours:',' ',h)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
False
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
False
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
False
Optimize a model with 9519 rows, 5166 columns and 21143 nonzeros
False
Model fingerprint: 0xf3da4ada
False
Variable types: 2583 continuous, 2583 integer (2583 binary)
False
Coefficient statistics:
False
  Matrix range     [2e-06, 1e+03]
False
  Objective range  [1e+00, 1e+00]
False
  Bounds range     [1e+00, 1e+00]
False
  RHS range        [1e-01, 4e+01]
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False

Presolve removed 6527 rows and 1662 columns
Presolve time: 0.02s
Presolved: 444 rows, 921 columns, 5243 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0632899e+06   1.808435e+04   0.000000e+00      0s
Extra simplex iterations after uncrush: 8
     565    3.7842854e+02   0.000000e+00   0.000000e+00      0s

Solved in 565 iterations and 0.06 seconds
Optimal objective  3.784285399e+02
***CHECK***
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6971 rows, 2583 columns and 15349 nonzeros
Model fingerprint: 0x0e7a2181
Coefficient statistics:
  Matrix range     [2e-06, 5e+01]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e+03]
Presolve removed 0 rows and 80 columns
Presolve time: 0.01s

Solved i

  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+03]
Presolve removed 6527 rows and 1662 columns
Presolve time: 0.02s
Presolved: 444 rows, 921 columns, 5239 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0263019e+06   1.786710e+04   0.000000e+00      0s
Extra simplex iterations after uncrush: 9
     612    4.8300367e+02   0.000000e+00   0.000000e+00      0s

Solved in 612 iterations and 0.05 seconds
Optimal objective  4.830036667e+02
***CHECK***
False
False
False
True
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6971 rows, 2583 columns and 15349 nonzeros
Model fingerprint: 0x30472488
Coefficient statistics:
  Matrix range     [2e-06, 5e+01]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+03]
Presolve removed 6528 rows and 1663 columns
Presolv

Optimal objective  4.816893834e+02
***CHECK***
False
False
False
True
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6971 rows, 2583 columns and 15349 nonzeros
Model fingerprint: 0x0428cd19
Coefficient statistics:
  Matrix range     [2e-06, 5e+01]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+03]
Presolve removed 6527 rows and 1662 columns
Presolve time: 0.02s
Presolved: 444 rows, 921 columns, 5242 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0632899e+06   1.739685e+04   0.000000e+00      0s
Extra simplex iterations after uncrush: 11
     674    4.8110623e+02   0.000000e+00   0.000000e+00      0s

Solved in 674 iterations and 0.06 seconds
Optimal objective  4.811062284e+02
***CHECK***
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Gurobi Opt

  RHS range        [1e-02, 1e+03]
Presolve removed 6527 rows and 1662 columns
Presolve time: 0.03s
Presolved: 444 rows, 921 columns, 5243 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0632899e+06   1.745599e+04   0.000000e+00      0s
Extra simplex iterations after uncrush: 15
     646    2.6123658e+02   0.000000e+00   0.000000e+00      0s

Solved in 646 iterations and 0.06 seconds
Optimal objective  2.612365764e+02
***CHECK***
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6971 rows, 2583 columns and 15349 nonzeros
Model fingerprint: 0xe00a6d2e
Coefficient statistics:
  Matrix range     [2e-06, 5e+01]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+03]
Presolve removed 6528 rows and 166

In [10]:

print('Biomass')

print('WT:','-->',FBA_WT)

print('Inner values Vij:','-->',vis[bm_core])
try:
    print('Outer Values Voj:','-->',voj[bm_core])
except:
    print('There are no feasible outer values')
print('Values from Cobra:','-->',vjc[bm_core])

print('***By-products***')

for i in in_target:
    print('%s WildType'%rxn[i],'--:',fba[i])
    print('%s COBRA values'%rxn[i],'--:',vjc[i])
    try:
        print('%s Overproduction'%rxn[i],'--:',voj[i])
    except:
        print('There are no feasible outer values')
    print('%s Inner values'%rxn[i],'--:',vis[i])

    print('*****')

print('Deletion strategy:')
    
print('Index','->','Reaction','->','Name','->','Y value')

try:
    for i in range(len(rxn)):
        if yoj[i] < .5:
            print(i,'->',rxn[i],'->',names[i],'-->',yoj[i])
except:
    print('There are no yj values')
    
yjc = [1 for i in range(len(rxn))]

yjc[k1] = 0

yjc[k2] = 0

print('Solution from COBRA')
for i in range(len(rxn)):
    if yjc[i] < .5:
        print(i,'->',rxn[i],'->',names[i],'-->',yjc[i])

Biomass
WT: --> 0.2415018333643431


NameError: name 'vis' is not defined

In [ ]:
voj[chemical]